In [1]:
from dotenv import load_dotenv
import os
apikey = os.getenv("PINECONE_API_KEY")

In [2]:
# !pip install -qU datasets

# Create client

In [3]:
from pinecone import Pinecone

pinecone_client = Pinecone(api_key=apikey)
index_name = "wiki"

# Pinecone에 있는 모든 인덱스를 순회합니다.
for idx in pinecone_client.list_indexes():
    # 인덱스 이름이 "wiki"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pinecone_client.delete_index(idx.name)
from pinecone import ServerlessSpec
quick_index = pinecone_client.create_index(
    name=index_name,
    dimension=1536,  # 모델 차원
    metric="cosine",  # 모델 메트릭
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [4]:
from datasets import load_dataset

data = load_dataset("wikimedia/wikipedia", "20231101.ko", split='train[:100]')

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data[0]

{'id': '5',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으

In [6]:
for lec in data:
    if len(lec['text'])>35000:
        print('@@')

@@
@@


# Embedding

In [7]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model= 'text-embedding-3-small')
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000027D4342A4B0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000027D66E98B00>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter:
# - 긴 텍스트를 지정한 크기(chunk_size) 단위로 나눠주는 유틸
# - 단순히 자르는 게 아니라, 주어진 'separators' 기준으로 먼저 분할을 시도하고
#   그래도 길면 더 작은 단위로 재귀적으로 나누는 방식

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,   # 한 덩어리(청크)의 최대 길이. (여기서는 400자 단위로 자름)
    chunk_overlap = 20, # 청크 사이에 겹치는 부분 길이. (문맥 유지용, 앞/뒤 20자 겹치게)
    length_function = len, \
    # 텍스트 길이를 재는 함수. 여기서는 파이썬 내장 len() → 문자열 길이 그대로 사용.
    # 필요에 따라 토큰 카운터로 바꿀 수도 있음 (예: tiktoken 길이 기준).

    separators=['\n\n', '\n', ' ', '']  
    # 분할 기준 우선순위 리스트.
    # 1) 두 줄 개행(\n\n) 단위로 먼저 잘라보고,
    # 2) 안 되면 한 줄 개행(\n),
    # 3) 그래도 안 되면 공백(' '),
    # 4) 마지막엔 글자 단위('')로 잘라서라도 chunk_size 이하로 맞춤.
)

splitter  # 설정된 splitter 객체 확인


In [12]:
import json
def calculate_metadata_size(metadata):
    return len(json.dumps(metadata, ensure_ascii=False).encode('utf-8'))

In [14]:
from uuid import uuid4
import time
texts =  []
metas = []
count = 0
batch_size = 20

In [17]:
# 'wiki' 인덱스를 가져옵니다.
index = pinecone_client.Index(index_name)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [18]:
for i, sample in enumerate(data):
    full_text = sample['text']
    
    #  메타데이터 구성
    metadata = {
        'wiki_id' : str(sample['id']),
        'url' : sample['url'],
        'title' : sample['title'] # [:1000]
    }
    
    # splitter
    chunks = splitter.split_text(full_text)
    # print(len(chunk))

    for i, chunk in enumerate(chunks):
        record = {
            'chunk_id':i,
            'full_text': full_text,
            **metadata
        }
        metadata_size = calculate_metadata_size(record)
        if metadata_size > 35000:
            continue
        
        texts.append(chunk)
        metas.append(record)
        count += 1
        
        if count % batch_size == 0:
            ids  = [str(uuid4())for _ in range(len(texts))]
            embeddings = embedding.embed_documents(texts) # 20개 청크의 수행
            index.upsert(
                vectors=zip(ids, embeddings, metas),
                namespace='wiki-ns1'
            )
            texts = []
            metas = []
            time.sleep(1)

# 질의 임베딩

In [19]:
question = ['벨기에는 어디에 있나요']
emb_question = embedding.embed_documents(question)

In [25]:
# 파인콘 질의 실행
q_result=index.query(
    namespace= 'wiki-ns1',
    vector=emb_question,
    top_k=5,
    include_metadata=True,
    include_values=True
)

In [26]:
q_result

{'matches': [{'id': 'ce7c3c93-72c2-468f-8e2a-ff6820ef0a28',
              'metadata': {'chunk_id': 11.0,
                           'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 '
                                        '없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. '
                                        '통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 '
                                        '로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, '
                                        '열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 '
                                        '루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 '
                                        '라틴 대문자 S를 쓴다.\n'
                                        '\n'
                                        '정의 \n'
                                        '엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 '
                                        '정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. '
                                        '고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 '
                                      

In [27]:
result_ids = [r.id for r in q_result.matches]
result_ids

['ce7c3c93-72c2-468f-8e2a-ff6820ef0a28',
 '82e5356c-8bc1-4220-bdd1-19bfa1d15332',
 '5e3a35f4-e88f-4b09-8301-6b0c7581d48d',
 '9a0b8626-e0cc-43f5-b37e-27f2a8e4750a',
 '7ad168f8-8a0b-4a6b-a6f9-ce25572bcf71']

# 랭체인의 api로 결과 비교하기

In [30]:
!pip install langchain_pinecone


   ---------------------------------------- 3/3 [langchain_pinecone]



In [31]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=question[0], k=5, namespace='wiki-ns1')
docs

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


[Document(id='ce7c3c93-72c2-468f-8e2a-ff6820ef0a28', metadata={'chunk_id': 11.0, 'title': '엔트로피', 'url': 'https://ko.wikipedia.org/wiki/%EC%97%94%ED%8A%B8%EB%A1%9C%ED%94%BC', 'wiki_id': '158'}, page_content='엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하

In [ ]:
question=[
    '엔트로피가 뭐야?',
    '벨기에 수도는?',
    '벡터는 언제 처음 만들어진 개념?'
]

In [33]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=question[0], k=5, namespace='wiki-ns1')
docs

[Document(id='ce7c3c93-72c2-468f-8e2a-ff6820ef0a28', metadata={'chunk_id': 11.0, 'title': '엔트로피', 'url': 'https://ko.wikipedia.org/wiki/%EC%97%94%ED%8A%B8%EB%A1%9C%ED%94%BC', 'wiki_id': '158'}, page_content='엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하

In [34]:
emb_question = embedding.embed_documents(question)

# 파인콘 질의 실행
q_result=index.query(
    namespace= 'wiki-ns1',
    vector=emb_question,
    top_k=5,
    include_metadata=True,
    include_values=True
)

q_result

{'matches': [{'id': 'ce7c3c93-72c2-468f-8e2a-ff6820ef0a28',
              'metadata': {'chunk_id': 11.0,
                           'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 '
                                        '없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. '
                                        '통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 '
                                        '로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, '
                                        '열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 '
                                        '루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 '
                                        '라틴 대문자 S를 쓴다.\n'
                                        '\n'
                                        '정의 \n'
                                        '엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 '
                                        '정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. '
                                        '고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 '
                                      

# Augmentation

In [35]:
# retriver
country  = '벨기에 '
query = f'{country}여행을 위한 관광지와 문화'
docs = vector_store.similarity_search(query=query, k=5, namespace='wiki-ns1')

In [36]:
context = ""
for result in docs:
    context += result.page_content

In [37]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import langchain 
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000027D620A2450>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027D69119C70>, root_client=<openai.OpenAI object at 0x0000027D68A5CC50>, root_async_client=<openai.AsyncOpenAI object at 0x0000027D620A6BA0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [40]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', '당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행일정 수립을 도와드립니다. 제한된 정보라도 최대한 활용을 하세요'),
        ('human', ''' {country}로 여행을 계획 중입니다. 
         아래 context에서 {country} 관련 정보를 찾아 여행 조언을 해주세요 
         만약 직접적인 정보가 없다면, 수도나 관련정보를 활용해주세요
         context: 
         {context}''')
    ]
)
chat_template

ChatPromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행일정 수립을 도와드립니다. 제한된 정보라도 최대한 활용을 하세요'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, template=' {country}로 여행을 계획 중입니다. \n         아래 context에서 {country} 관련 정보를 찾아 여행 조언을 해주세요 \n         만약 직접적인 정보가 없다면, 수도나 관련정보를 활용해주세요\n         context: \n         {context}'), additional_kwargs={})])

In [41]:
prompt =  chat_template.format_messages(country=country, context=context)
print(prompt)

[SystemMessage(content='당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행일정 수립을 도와드립니다. 제한된 정보라도 최대한 활용을 하세요', additional_kwargs={}, response_metadata={}), HumanMessage(content=' 벨기에 로 여행을 계획 중입니다. \n         아래 context에서 벨기에  관련 정보를 찾아 여행 조언을 해주세요 \n         만약 직접적인 정보가 없다면, 수도나 관련정보를 활용해주세요\n         context: \n         아쿠타가와 류노스케(, 1892년 3월 1일~1927년 7월 24일)는 일본의 소설가이다. 호는 징강당주인(澄江堂主人)이며 하이쿠 작가로서의 호는 가키(我鬼)이다.\n\n그의 작품은 대부분이 단편 소설이다. 「참마죽」, 「덤불 속」, 「지옥변」 등 주로 일본의 《곤자쿠 이야기집》·《우지슈이 이야기》 등 전통적인 고전들에서 제재를 취하였다. 또한 「거미줄(원제: 蜘蛛の糸)」, 「두자춘(杜子春)」 등 어린이를 위한 작품도 남겼으며, 예수를 학대한 유대인이 예수가 세상에 다시 올 때까지 방황한다는 상상력을 발휘한 「방황하는 유대인」도 있다.\n\n생애\n\n유년 시절 \n1892년(메이지 25년) 3월 1일 도쿄에서 우유 판매업자였던 아버지 니하라 도시조(新原敏三)와 어머니 후쿠(フク) 사이의 아들로 태어났다(아쿠타가와라는 성은 원래 그의 어머니쪽 성씨였다). 이때 태어난 시간이 공교롭게도 진년(辰年) 진월(辰月) 진일(辰日) 진시(辰時)였기 때문에 \'용(龍)\' 자를 이름에 넣어 류노스케(龍之介)라 짓게 되었다고 전하나, 실제 그가 태어난 1892년 3월 1일은 간지로는 임진년·임인월·임진일에 해당하며, 출생 시각에 대해서는 자료가 없기 때문에 확실한 것이 없다. 이름도 호적상으로는 \'龍之介\'이지만 그가 양자로 들어갔던 아쿠타가와 집안이나 졸업한 학교의 명단 등의 문서에는 \'龍之助\'로 되어 있다(아쿠타가와 자신은 \'龍之助\

In [42]:
response = llm.invoke(prompt)
response.content

'벨기에 여행을 계획하며 몇 가지 중요한 정보를 제공하겠습니다.\n\n### 1. 수도: 브뤼셀\n브뤼셀은 벨기에의 수도이자 유럽 연합의 중심지입니다. 정치적, 문화적 활동의 중심으로, 다양한 역사적인 건축물과 명소가 있습니다. 특히, **그랑플라스**와 **아토미움**은 꼭 방문해야 할 곳입니다.\n\n### 2. 음식\n벨기에는 맥주와 초콜릿으로 유명합니다. 벨기에의 **수제 맥주**를 경험하고, **와플**과 **프리츠(튀긴 감자)**를 현지에서 맛보세요. 또한, 벨기에 초콜릿 가게를 방문해 다양한 초콜릿을 시식하는 것도 추천합니다.\n\n### 3. 교통\n벨기에의 도시들은 기차로 쉽게 연결되어 있습니다. **유로스타**나 **Thalys**와 같은 고속 기차를 이용하면 브뤼셀과 유럽의 다른 도시들 (파리, 암스테르담, 런던 등) 간에 편리하게 이동할 수 있습니다.\n\n### 4. 문화 및 예술\n브뤼셀에서는 매년 다양한 문화 행사 및 음악 축제가 열립니다. 예술작품을 좋아하신다면 **마그리트 미술관**이나 **벨기에 왕립미술관**을 방문해보세요. 이러한 장소에서는 벨기에의 유명 화가들의 작품과 현대 미술을 관람할 수 있습니다.\n\n### 5. 기후\n벨기에는 일반적으로 온화한 기후를 가지고 있습니다. 여름철에는 따뜻하지만, 가끔 비가 내릴 수 있으니 우산이나 우비를 챙기는 것이 좋습니다. \n\n### 6. 여행 일정 팁\n- **1일차**: 브뤼셀 도착 후 그랑플라스, 아토미움 방문\n- **2일차**: 브뤼주로 이동하여 운하 투어 및 중세 건축물 탐방\n- **3일차**: 겐트 방문, 성당과 성 탐방 후 맥주 시음\n- **4일차**: 앤트워프에서 다이아몬드 투어와 미술관 방문\n\n이 일정은 기본적인 여정으로, 고객의 관심사에 따라 조정 가능하니 더 구체적인 질문이나 선호하시는 활동이 있다면 알려주세요. 즐거운 여행이 되시길 바랍니다!'

In [ ]:
# 벡터 검색 + 키워드 필터링
def enhanced_search(country, vector_store):
    # 1단계: 벡터 검색
    vector_results = vector_store.similarity_search(
        query=f"{country} 여행 관광", 
        k=10, 
        namespace="wiki-ns1"
    )
    
    # 2단계: 국가명이 포함된 결과 우선순위
    country_related = []
    other_results = []
    
    for result in vector_results:
        if country.lower() in result.page_content.lower() or \
           "벨기에" in result.page_content or \
           "브뤼셀" in result.page_content:
            country_related.append(result)
        else:
            other_results.append(result)
    
    # 관련성 높은 결과 우선 반환
    final_results = (country_related + other_results)[:5]
    return final_results

results = enhanced_search("벨기에", vector_store)
results